# Welcome to my Capstone Project

## Box Office Analyzer

### About this project:

This program will allow the end user to analyze a period of box office results.  They will be able to look at things like, "Unique Films in Top 5", "Gross of New Openers During Period", "Total Box Office by Week during Period", etc.

The program will be interactive, meaning that we will start by asking the user what period (start and end date) that they would like to analyze and then give them option that provide back various statistics.

### Here is what we need to do to accomplish this.

1.  Check to see if a BoxOfficeDB.csv exists in the Data folder.
    If it does not, this is the first program run, so, create it with the appropriate headers
1. Read in the BoxOfficeDB.csv database file.
1. Ask the user to specify the Start Date and End Date to Analyze (allow for flexible date entry).
1. Create a Range with each unique Friday that is in the Analysis Period
1. Check to see which (if any) of those Fridays are already in the BO Database.
1. For the Fridays that we don't already have, fetch them from the-numbers.com and add them to the Box Office DB.
1. Save the updated Box Office DB for the future.  This will save on the number of times that we have to go to the-numbers.com and use their resources.
1. Present the user with the analytics that are supported by the program and ask them to choose.  Give the option to Quit.
1. Perform the analysis.
1. Repeat previous two steps until they Quit.


### Program Flow

import modules

create global variables:

class BoxOfficeEntry():
    
    self.friday_date
	self.bo_year
    self.bo_month
    self.bo_day
    self.bo_location
    self.title_id
    self.rank
    self.previous_rank
    self.title
    self.distributor
    self.distributor_id
    self.gross
    self.change_pct
    self.theaters
    self.per_theater
    self.total_gross
    self.weeks_in_release
    self.created_on 

class BoxOfficeAnalyzer():

    df_box_office_db = pd.DataFrame()
    
    def __init__(start_date, end_date):
        '''
        NOTE: On initialization, we will perform the following:
        1. Check for existings database
        2. Create one if it doesn't exist
        3. Read the database into a DataFrame
        4. Download missing weeks in the period that has been specified using the start_date and end_date
        5. Add them to the DataFrame
        6. Save the DataFrame as the updated boxofficedb.csv file for future runs.

        When you are ready to perform analysis, call the box_office.which_analysis() function.
    '''
    def add_box_office_entry(box_office_entry):
    
    def check_for_boxoffice_db():

    def create_boxoffice_db():

    def get_missing_bo_weekends():
    
    def which_analysis():
    
    def perform_analysisX():
    
    
def get_analysis_period():
    return start_date, end_date

def main():

    print("Welcome Box Office fans!")
    start_date, end_date = get_analysis_period()
    
    bo_analyzer = BoxOfficeAnalyzer(start_date, end_date)
    
    response = ""
    
    while response[0] not in ("q", "Q"):
        
        response = input("Would you like to perform another analysis? (y/q)")
        
        if response.lower() == "y":
            
            bo_analyzer.which_analysis()


    print("Thanks and see you at the movies!"


In [28]:
# Author: Jason Bubenicek
# Version: 0.1

In [1]:
# %matplotlib inline

# Modules
import time
import os
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML

In [2]:
# Global Variables
BOX_OFFICE_DB = './Data/BoxOfficeDB.csv'
BASE_URL = 'https://www.the-numbers.com/box-office-chart/weekend'
DEFAULT_START_DATE = "12/21/2018"
DEFAULT_END_DATE = "12/28/2018"
LOCATION = "Domestic"

In [27]:
# Class Definitions

class BoxOfficeEntry():
    '''
    Usage:
    As we fetch a row of weekend box office from
    the-numbers.com, create a BoxOfficeEntry.  A typical
    use will be to add it to the Box Office database, by
    passing it to a BoxOfficeAnalyzer.add_box_office_entry()
    method call
    
    Note:
    There are no variable to pass when instanciating this object.
    You will need to set each property after setup.
    '''

    friday_date        = None
    bo_year            = None
    bo_month           = None
    bo_day             = None
    bo_location        = None
    title_id           = None
    rank               = None
    previous_rank      = None
    title              = None
    distributor        = None
    distributor_id     = None
    gross              = None
    change_pct         = None
    theaters           = None
    per_theater        = None
    total_gross        = None
    weeks_in_release   = None
    created_on         = None
    
class BoxOfficeAnalyzer():
    '''
    Usage: This class will handle most everything that you need
    to perform various analysis, including fetching the data to
    be analyzed.  In most cases, you will only need a single copy
    of this class instanciated.
    '''
    
    # Create a Data Frame to store our Box Office Database.
    df_box_office_db = pd.DataFrame
    df_box_office_subset = pd.DataFrame
    
    def __init__(self, start_date, end_date):
        try:
            self.start_date = datetime.strptime(start_date, "%m/%d/%Y")
            self.end_date = datetime.strptime(end_date, "%m/%d/%Y")
        except ValueError:
            print("You did not pass a start or end date in m/d/Y format, setting default Start ({DEFAULT_START_DATE}) and End ({DEFAULT_END_DATE}) dates")
            self.start_date = datetime.strptime(DEFAULT_START_DATE, "%m/%d/%Y")
            self.end_date = datetime.strptime(DEFAULT_END_DATE, "%m/%d/%Y")
            
        if not self.check_for_boxoffice_db():
            print("Box Office DB does NOT exist.  Creating one...")
            
            if not self.create_boxoffice_db():
                print("Failed to create Box Office DB.\nThe program will exit.")
        
        if not self.read_box_office_database():
            print("Failed to load Box Office DB\nThe program will exit.")
            
        if not self.get_missing_bo_weekends():
            print("Failed to get missing Box Office Weekends.\nThe program will exit.")

        if not self.save_box_office_db():
            print("Failed to save Box Office DB...")
            
        if not self.__format_db():
            print("Failed to add Weekend Date column...")
                    
        rows, cols = self.df_box_office_db.shape
        print(f"Database details: Columns: {cols}, Rows: {rows}.")

        if not self.__filter_db():
            print("Failed to filter Box Office DB...")

        
        
        print("Ready for Analytics...")
    
    def __format_db(self):
        self.df_box_office_db[self.df_box_office_db.columns[8:9]] = self.df_box_office_db[self.df_box_office_db.columns[8:9]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
        self.df_box_office_db['Weekend_Date_str'] = self.df_box_office_db['bo_month'].astype(str) + "/" + self.df_box_office_db['bo_day'].astype(str) + '/' + self.df_box_office_db['bo_year'].astype(str)
        self.df_box_office_db["Weekend_Date_date"] = pd.to_datetime(self.df_box_office_db["Weekend_Date_str"], infer_datetime_format=True)
        self.df_box_office_db['gross$']=self.df_box_office_db['gross'].apply(self.format_bo)
        # self.df_box_office_db.columns = ['Day of Month', 'Location', 'Month', 'Year', 'Change %', 'Created On']
        
        return True
        
    def __filter_db(self):
        
        #df['Weekend_Date_str'].head()

        #df["Weekend_Date_date"] = pd.to_datetime(df["Weekend_Date_str"], infer_datetime_format=True)

        # df[["Weekend_Date_date", "title", "gross"]]
        # pd.DataFrame(df.loc[(df['Weekend_Date_date'] >= datetime(2018, 12,15) )]).head()
        df_filtered = pd.DataFrame(self.df_box_office_db.loc[(self.df_box_office_db['Weekend_Date_date'] >= self.start_date) & (self.df_box_office_db['Weekend_Date_date'] <= self.end_date)])
        df_filtered.head()
        
        rows, cols = df_filtered.shape
        print(f"Filtered Database details: Columns: {cols}, Rows: {rows}.")
        
        self.df_box_office_db = df_filtered
        
        return True
    
    
    def add_box_office_entry(self, box_office_entry):
        print("Added Entry")
        
    def read_box_office_database(self):
        print("Loading Box Office database...")
        self.df_box_office_db = pd.read_csv(BOX_OFFICE_DB)
        return True

    def save_box_office_db(self):
        print("Saving Box Office database...")
        try:
            self.df_box_office_db.to_csv(BOX_OFFICE_DB, index=False)
        except Exception as e:
            print(f"Exception: {e}")
            return False
        
        return True
        
    def check_for_boxoffice_db(self):
        print("Checking for existing Box Office Database in the Data folder...")
        return os.path.isfile(BOX_OFFICE_DB)
            
    def create_boxoffice_db(self):
        print("Creating a blank Box Office database...")
        date_to_fetch = self.start_date
            
        try:
            self.df_box_office_db = self.get_bo_weekend(date_to_fetch, verbose=False)
            if not self.save_box_office_db():
                return False
        except Exception as e:
            print(f"Exception: {e}")
            return False

        return True

                
                
        return True

    def get_missing_bo_weekends(self):
        print("Getting missing Box Office Weekends from 'the-numbers.com'...")
        
        unique_dates = pd.DataFrame(self.df_box_office_db, columns=['bo_year','bo_month', 'bo_day'])
        unique_dates.drop_duplicates(inplace=True)
        unique_dates['Weekend_Date_str'] = unique_dates['bo_month'].astype(str) + "/" + unique_dates['bo_day'].astype(str) + '/' + unique_dates['bo_year'].astype(str)
        unique_dates["Weekend_Date_date"] = pd.to_datetime(unique_dates["Weekend_Date_str"], infer_datetime_format=True)
        # unique_dates.head()
        
        for week in range(0, self.__get_num_weeks()+1):
            days_to_add = (week * 7)
            date_to_fetch = self.start_date  + timedelta(days=days_to_add)
            # print(f"Add {days_to_add} Days...")
            # print(self.start_date  + timedelta(days=days_to_add))
            print(f"Checking Box Office for {str(date_to_fetch)}.")
            if pd.DataFrame(unique_dates.loc[(unique_dates['Weekend_Date_date'] == date_to_fetch)])['bo_year'].count() == 0:
                print(f"Fetching Missing Box Office for {str(date_to_fetch)}...")
                
                try:
                    df_missing_bo_weekend = self.get_bo_weekend(date_to_fetch, verbose=False)

                    if self.__append_bo(df_missing_bo_weekend):
                        print(f"Appended Missing Box Office for {str(date_to_fetch)}...")
                    else:
                        print(f"Failed to Append Missing Box Office for {str(date_to_fetch)}...")
                    
                except Exception as e:
                    print(f"Failed to fetch Box Office for {str(date_to_fetch)}\n{e}.")



        # print(f"Number of Weeks: {str(self.__get_num_weeks())}")

        return True

    def __append_bo(self, df_bo_weekend):
        
        try:
            self.df_box_office_db = pd.concat([self.df_box_office_db, df_bo_weekend], sort=True, ignore_index=True)
            rows, cols = self.df_box_office_db.shape
            print(f"Database details: Columns: {cols}, Rows: {rows}.")
            
            # self.df_box_office_db.append(df_bo_weekend, sort=False, ignore_index=True)
            
        except Exception as e:
            print(f"Exception: {e}")
            return False
        
        return True
    
    def __get_num_weeks(self):
        friday1 = (self.start_date - timedelta(days=self.start_date.weekday()))
        friday2 = (self.end_date - timedelta(days=self.end_date.weekday()))

        return int(round((friday2 - friday1).days / 7))
        
    def get_bo_weekend(self,friday_date=datetime.strptime(DEFAULT_START_DATE, "%m/%d/%Y"), verbose=False):
        
        # Example URL                                    
        # https://www.the-numbers.com/box-office-chart/weekend/2018/11/23

        url = BASE_URL + '/' + str(friday_date.year) + "/" + str(friday_date.month).zfill(2) + "/" + str(friday_date.day).zfill(2)

        page_test = "Weekend Domestic Chart for "
        bo_weekend_parts = str(url).split('/')
        bo_year = bo_weekend_parts[5]
        bo_month = bo_weekend_parts[6]
        bo_day = bo_weekend_parts[7]
        bo_location = LOCATION

        page_response = requests.get(url, timeout=5)   

        df_boxoffice = pd.DataFrame()


        if page_response.status_code == 200:
            page_content = BeautifulSoup(page_response.content, "html.parser")    

            # Confirm that we have the right content before proceeding
            # some websites will redirect you when you attempt to access
            # a page that results in a error or does not exist.
            if str(page_content.find(attrs={'id': 'main'})).find(page_test) != -1:

                # Rank, Last Rank 	Movie	Distributor	Gross	Change	Thtrs.	Per Thtr.	Total Gross	Week
                #1	new	Night School	Universal	$27,257,615	 	3,010	$9,056	  $27,257,615	1

                for table_row in page_content.select("div#page_filling_chart center table tr"):
                    # Each tr (table row) has three td HTML elements (most people
                    # call these table cels) in it (first name, last name, and age)
                    cells = table_row.findAll('td')


                    if cells:
                        title_url_a = cells[2].select("b a")
                        distributor_url_a = cells[3].select("a")

                        rank = cells[0].text.strip()
                        previous_rank = cells[1].text.strip()
                        title = cells[2].text.strip()
                        title_id= str(str(title_url_a[0].get('href')).replace("#tab=box-office","").replace("/movie/",""))
                        distributor = cells[3].text.strip()
                        distributor_id= str(distributor_url_a[0].get('href')).replace("/market/distributor/","")
                        gross = cells[4].text.strip()
                        change_pct = cells[5].text.strip()
                        theaters = cells[6].text.strip()
                        per_theater = cells[7].text.strip()
                        total_gross = cells[8].text.strip()
                        weeks_in_release = cells[9].text.strip()

                        if verbose:
                            print("{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{11},{12},{13},{14}".format(
                                bo_year
                                , bo_month
                                , bo_day
                                , bo_location
                                , title_id
                                , rank
                                , previous_rank
                                , title
                                , distributor
                                , distributor_id
                                , gross
                                , change_pct
                                , theaters
                                , per_theater
                                , total_gross
                                , weeks_in_release
                                , datetime.now()))

                        df_row = pd.DataFrame([[
                            bo_year
                            , bo_month
                            , bo_day
                            , bo_location
                            , title_id
                            , rank
                            , previous_rank
                            , title
                            , distributor
                            , distributor_id
                            , gross
                            , change_pct
                            , theaters
                            , per_theater
                            , total_gross
                            , weeks_in_release
                            , datetime.now()]], 
                            columns=[
                            'bo_year'
                            , 'bo_month'
                            , 'bo_day'
                            , 'bo_location'
                            , 'title_id'
                            , 'rank'
                            , 'previous_rank'
                            , 'title'
                            , 'distributor'
                            , 'distributor_id'
                            , 'gross'
                            , 'change_pct'
                            , 'theaters'
                            , 'per_theater'
                            , 'total_gross'
                            , 'weeks_in_release'
                            , 'created_on'
                            ])

                        df_boxoffice = df_boxoffice.append(df_row)

        return df_boxoffice

    def which_analysis(self):
        
        response = 0
        reports = 5
        while response not in range(1,reports+1):
            
            # Reminder: range is up to, not including the last number.  So, this is will return 1, 2.
            try:
                print("Which analysis would you like to perform:")
                print("1. Unique Films in the Top 10 During Period")
                print("2. Total Weekend Box Office Per Week")
                print("3. Top 10 Results During Period")
                print("4. Top 5 Films")
                print("5. Top 10 Distributors")
                response = int(input(f"\nEnter a number between 1 and {reports}..."))
                
            except:
                print("You must enter a number.  Please try again.")
        
        # analysis_functions = [self.perform_analysis_0, self.perform_analysis_1, self.perform_head]
        # print(analysis_functions[response - 1]())

        if response == 1:
            self.unique_titles_in_top10()
        elif response == 2:
            self.total_bo_by_weekend()
        elif response == 3:
            self.top10_weekend_results()
        elif response == 4:
            self.top5_during_period()
        elif response == 5:
            self.top10_distribs()
        else:
            print("Unknown Analysis selected...")
        
    def unique_titles_in_top10(self):
        unique_titles = pd.DataFrame(self.df_box_office_db, columns=['Weekend_Date_date', 'title', 'gross'])
        unique_titles.drop_duplicates(inplace=True)
        #unique_titles[unique_titles.columns[2:]] = unique_titles[unique_titles.columns[2:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
        
        df_sum = unique_titles[['title', 'gross']].groupby(['title']).sum()
        df_sum.sort_values(['gross'], ascending=False, inplace=True)
        df_sum['gross$']=df_sum['gross'].apply(self.format_bo)
        
        # print(df_sum[['gross$']].head(10))
        df_top10 = df_sum.head(10)

        df_top10 = df_sum.head(10)
        df_top10.columns = ['Gross', 'Gross ($)']
        display(HTML(df_top10[['Gross ($)']].to_html()))
    
    def total_bo_by_weekend(self):
        df = pd.DataFrame(self.df_box_office_db, columns = ['Weekend_Date_date', 'gross'])
        #df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
        df_sum = df.groupby(['Weekend_Date_date']).sum()
        df_sum.sort_values(['Weekend_Date_date'], ascending=True, inplace=True)
        df_sum['gross$']=df_sum['gross'].apply(self.format_bo)
        
        # ax = df_sum.plot.bar()



        # print(df_sum[['gross$']].head(10))
        df_top10 = df_sum.head(10)
        df_top10.columns = ['Gross', 'Gross ($)']
        display(HTML(df_top10[['Gross ($)']].to_html()))
    
    def format_bo(self, x):
        return "${:.1f}m".format((x/1000000))
    
    def top5_during_period(self):
        
        df = pd.DataFrame(self.df_box_office_db, columns = ['Weekend_Date_date', 'title', 'gross'])
        #df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
        df["gross"] = pd.to_numeric(df["gross"])
        df.sort_values(['gross'], ascending=False, inplace=True)
        df['gross$']=df['gross'].apply(self.format_bo)
        df_top5 = df.head(5)
        df_top5.columns = ['Weekend', 'Title', 'Gross', 'Gross ($)']

        display(HTML(df_top5.to_html(index=False)))
        

        #ax = df_top5.sort_values(['Gross'], ascending=True).plot.barh(x='Title', y='Gross', rot=0)
        #ax.set_xlabel("Box Office (USD/m)")
        #ax.set_ylabel("Films")
        
    
    def top10_weekend_results(self):
        print("Performing head()..")
        df = pd.DataFrame(self.df_box_office_db, columns = ['Weekend_Date_date', 'title', 'gross'])
        #df[df.columns[2:]] = df[df.columns[2:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
        df.sort_values(['gross'], ascending=False, inplace=True)
        
        df_top10 = df.head()
        df_top10.columns = ['Weekend', 'Title', 'Gross ($)']
        
        # TODO: Work on formatting / Left Justify Strings and Currency Formatting.
        # df_top10.style.format({"Gross ($)": "${:,.2f}"})
        #df_top10.style.apply('text-align: left', subset=['Title'])

        # df_top10["Gross ($)"] = df_top10['Gross ($)'].map('${:,.2f}'.format)
        # df_top10['Gross ($)'].map('${:,.2f}'.format)
        
        # TODO: Resolve copy warning
        df_top10.loc[:,'Gross ($)'] = df_top10['Gross ($)'].map('${:,.2f}'.format)
        
        #df_top10 = df_top10.style.format({'Gross ($)':'${:,.2f}'})
        
        # df_top10['Gross ($)'].applymap("${0:.2f}".format)
        display(HTML(df_top10.to_html(index=False)))
        
    def top10_distribs(self):
        df = pd.DataFrame(self.df_box_office_db, columns = ['distributor', 'gross'])
        #df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
        df_sum = df.groupby(['distributor']).sum()
        df_sum.sort_values(['gross'], ascending=False, inplace=True)
        df_sum['gross$']=df_sum['gross'].apply(self.format_bo)
        df_sum.columns = ['Distributor', 'Gross ($)']
        
        
        #print(df_sum[['Gross ($)']].head(10))
        df_top10 = df_sum.head(10)
        df_top10.set_index('Distributor')
        
        display(HTML(df_top10[['Gross ($)']].to_html()))

    def get_friday_date(self,test_date):
        return test_date
        

def validate_date(input_date, first=datetime(1980, 1, 1),
                              last=datetime.now(),
                              fmt='%m/%d/%Y'):
    """Return a validated datetime.datetime or None.

    If the date has the wrong format return None, or if it is not in
    the range [first,last] also return None.  Otherwise return the
    input_date as a datetime.datetime object.

    """
    try:
        d = datetime.strptime(input_date, fmt)
        if not (first <= d <= last):
            raise ValueError
    except ValueError:
        return None
    else:
        return d

def get_date(message):
    '''
    Usage: Pass a message to ask the user to specify a date in mm/dd/yyyy format.  When properly
    validated, will return a date in string format.
    '''
    
    my_date = ""
    
    while my_date == "":
        
        my_date = input(message)
        
        if my_date == "":
            my_date = DEFAULT_START_DATE
            
        valid_date = validate_date(my_date)
        if valid_date is not None:
            break
        else:
            print("The date must be a valid Friday between 1980 and today.  Please try again...")
            my_date = ""
    
    return my_date

def start():
    '''
    Usage:
    Call start() to start Analyzing Box Office.
    '''
    print("\n\n\nWelcome to the Box Office Analyzer.")
    print("\nUsing this application, you can fetch any period of Weekend Box")
    print("Office results and perform variable analysis on the data.")
    print("\n\nLet's get started...\n\n")
    time.sleep(.300)
    
    start_date = get_date("Enter the first Friday date in the analysis period (mm/dd/yyyy):")
    end_date = get_date("Enter the last Friday date in the analysis period (mm/dd/yyyy):")
        
    # boa = BoxOfficeAnalyzer(start_date="11/16/2018", end_date="12/28/2018")
    boa = BoxOfficeAnalyzer(start_date=start_date, end_date=end_date)
    
    ready = ""
    
    # Continue to ask if the player would like to play until
    # they indicate that they want to Quit the game.
    while ready.lower() not in ["q", "quit", "exit", "end"]:

        ready = input("Would you like to perform an analysis? ('Yes', 'No', 'Quit')")

        if ready.lower() in ["y", "yes"]:
            clear_output()
            boa.which_analysis()
            
            ready = ""


    print("Thanks and see you at the movies!") 


In [ ]:
start()